In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

from tslearn.clustering import TimeSeriesKMeans
from tslearn.datasets import CachedDatasets
from tslearn.preprocessing import TimeSeriesScalerMeanVariance, \
    TimeSeriesResampler

In [ ]:
def process_lit(path, year):
    lit=pd.read_excel(path).replace("-", "0").fillna(0)
    lit = lit.pivot(index="Semaine", columns="Code UF", values="Lits fermés moyens")
    lit['date_semaine'] = pd.date_range(start=f'{year}-01-01', periods=52, freq='W')
    lit.set_index('date_semaine', inplace=True)
    return lit

In [ ]:
lit_24= process_lit(r"C:\Stage\generated_data\lits\lit_2024.xlsx", 2024)
lit_23= process_lit(r"C:\Stage\generated_data\lits\lit_2023.xlsx", 2023)
lit= pd.concat([lit_23, lit_24], axis=0).fillna(0)
X_train = lit.T.values[:, :,np.newaxis]


In [ ]:
km = TimeSeriesKMeans(n_clusters=3, metric="dtw", random_state=0)
y_pred = km.fit_predict(X_train)


In [ ]:
plt.figure(figsize=(12, 4))
for yi in range(1,4):  # clusters 0, 1, 2
    plt.subplot(1, 3, yi + 1)
    for xx in X_train[y_pred == yi]:
        plt.plot(xx.ravel(), "k-", alpha=0.3)
    plt.title(f"Cluster {yi}")

plt.tight_layout()
plt.show()